# N-Gram Language Models

#### Preliminaries: Install KenLM

[KenLM](https://kheafield.com/code/kenlm/) is used in the latter part of this lab.

To install it, run the following commands **in the jupyter terminal** (see screenshot):

    sudo apt-get install build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev
    
    cd /home/jupyter
    wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
    mkdir kenlm/build
    cd kenlm/build
    cmake ..
    make -j 4
    
    cd /home/jupyter/kenlm
    python setup.py install
    
<img src="img/instruction.png" alt="Drawing" style="width: 35%;"/>

<img src="img/instruction2.png" alt="Drawing" style="width: 70%;"/>

In [1]:
!pip install jsonlines

In [2]:
%pylab inline
import os, sys, glob, json, math
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint
from collections import defaultdict
pd.set_option('display.max_colwidth', -1)

Populating the interactive namespace from numpy and matplotlib


### N-gram Language Modeling

In **language modeling** we want to model the probability of variable length sequences, $$\large p(w_1,\ldots,w_T)=\prod_{t=1}^T p(w_t|w_{<t}).$$

An **n-gram language model** assumes that each word $w_t$ only depends on the preceding $n-1$ words, $$\large p(w_1,\ldots,w_T)=\prod_{t=1}^T p(w_t|w_{t-n+1},\ldots,w_{t-1}).$$

 

#### Example
For instance, when modeling the sentence $$\texttt{the cat sat on the mat .}$$ a 3-gram language model assumes that $$p(\texttt{mat}|\texttt{the cat sat on the}) \approx p(\texttt{mat}|\texttt{on the}).$$

The sub-sequence $(\texttt{on the mat})$ is a *3-gram* or *trigram*.



### Count-based Estimation

Given some dataset $D$ of sequences, we can estimate an n-gram model through counting, derived as follows:

\begin{align}
p(w_t|w_{t-n+1},\ldots,w_{t-1}) &= \frac{p(w_{t-n+1},\ldots,w_t)}{p(w_{t-n+1},\ldots,w_{t-1})} & \text{definition of conditional probability}\\
                       &= \frac{p(w_{t-n+1},\ldots,w_t)}{\sum_{w_{t'}}p(w_{t-n+1},\ldots,w_{t-1},w_{t'})}\\
                       &\approx \frac{\frac{1}{N}\text{count}(w_{t-n+1},\ldots,w_t)}{\frac{1}{N}\sum_{w_{t'}}\text{count}(w_{t-n+1},\ldots,w_{t-1},w_{t'})}\\
                       &= \frac{\text{count}(w_{t-n+1},\ldots,w_t)}{\sum_{w_{t'}}\text{count}(w_{t-n+1},\ldots,w_{t-1},w_{t'})}\\
                       &= \frac{\text{count}(w_{t-n+1},\ldots,w_t)}{\text{count}(w_{t-n+1},\ldots,w_{t-1})},
\end{align}

where $N$ is the number of $n$-grams in the dataset.

In Python, we can collect these counts into a dictionary mapping a prefix to a dictionary of counts:

        count[(w_n+1,...,w_t-1)] = {wt1: count of (w_n+1,...,w_t1),
                                    wt2: count of (w_n+1,...,w_t2),
                                    ...
                                   }
                                   
and for the denominator, maintain a dictionary of totals:

        total[(w_n+1,...,w_t-1)] = count of w_n+1,...,w_t-1

### Simplified Language

To get intuition, lets start by modeling a simple language called `ABC`. A word in this language is one of three tokens, $$w\in \{\texttt{A, B, C}\},$$
and we'll denote a sentence as $\textbf{w}=(w_1,\ldots,w_{|\textbf{w}|})$.


Suppose we are given the following dataset, and want to estimate a **bigram model**: $$p(\textbf{w})\approx\prod_{t=1}^{|\textbf{w}|}p(w_t|w_{t-1})\quad\quad(*)$$

In [3]:
data_raw = [['A', 'A', 'B', 'B'],
            ['A', 'A', 'B'],
            ['A', 'A', 'B', 'C'],
            ['A', 'A', 'A'],
            ['A', 'A', 'A', 'A']]

Since our model is a probability distribution, the total probability of all possible strings in the language must sum to 1, i.e.: $$\sum_{\textbf{w}}p(\textbf{w})=1.$$

In order to satisfy this criterion it turns out that we need an additional **beginning token**, `<bos>`, and **end token**, `<eos>`:

In [4]:
data = [['<bos>'] + d + ['<eos>'] for d in data_raw]
data

[['<bos>', 'A', 'A', 'B', 'B', '<eos>'],
 ['<bos>', 'A', 'A', 'B', '<eos>'],
 ['<bos>', 'A', 'A', 'B', 'C', '<eos>'],
 ['<bos>', 'A', 'A', 'A', '<eos>'],
 ['<bos>', 'A', 'A', 'A', 'A', '<eos>']]

Now let's estimate a bigram model:

\begin{align}
p(w_t|w_{t-1}) &= \frac{\text{count}(w_{t-1}w_{t})}{\sum_{w_{t'}}\text{count}(w_{t-1}w_{t'})}\\
               &= \texttt{count[prefix][wt] / totals[prefix]}
\end{align} 

where $\texttt{prefix}$ is $w_{t-1}$ in this case.

In [5]:
count = defaultdict(lambda: defaultdict(float))
total = defaultdict(float)

n = 2
for sequence in data:
    for i in range(len(sequence)-n+1):         # for each ngram
        ngram = tuple(sequence[i:i+n])
        prefix, word = ngram[:-1], ngram[-1]
        count[prefix][word] += 1               # count(w_{t-n+1}...w_t)
        total[prefix] += 1                     # count(w_{t-n+1}...w_{t-1})

Let's see if the counts and totals make sense:

- How many times did (A, B) occur? What about (B, B)?
- How many times did (A) occur? What about (C)?

In [6]:
print("Counts:")
pprint(dict(count))
print("\nTotals:")
pprint(dict(total))

Counts:
{('<bos>',): defaultdict(<class 'float'>, {'A': 5.0}),
 ('A',): defaultdict(<class 'float'>, {'A': 8.0, 'B': 3.0, '<eos>': 2.0}),
 ('B',): defaultdict(<class 'float'>, {'B': 1.0, '<eos>': 2.0, 'C': 1.0}),
 ('C',): defaultdict(<class 'float'>, {'<eos>': 1.0})}

Totals:
{('<bos>',): 5.0, ('A',): 13.0, ('B',): 4.0, ('C',): 1.0}


#### Conditional probability queries

We can now query a conditional probability:

\begin{align}
\texttt{p(word|prefix)} =&\ \texttt{count[prefix][word] / totals[prefix]}
\end{align}

In [7]:
queries = [('<bos>', 'A'),
           ('B', 'C')]

for query in queries:
    prefix, word = query[:-1], query[-1]
    p = count[prefix][word] / total[prefix]  # We'll discuss the case when `total[prefix] = 0` below.
    print("p( %s | %s) = \t%.5f" % (word, ', '.join(prefix), p))

p( A | <bos>) = 	1.00000
p( C | B) = 	0.25000


**Exercise**: Look at the training set and convince yourself that these conditional probabilities are correct according to the count-based estimation procedure.

#### Sequence Probability

We can compute the probability of a sequence using the conditional probabilities along with the chain rule of probability:

\begin{align}
p(w_1,\ldots,w_T)&\approx\prod_{t=1}^T p(w_t|w_{t-1})
\end{align}

(Here $w_0$ is `<bos>` and $w_T$ is `<eos>`)

In [8]:
sequence = ['<bos>', 'A', 'A', 'B', '<eos>']

def sequence_p(sequence, log=False):
    total_p = 1

    for i in range(len(sequence)-n+1):
        ngram = tuple(sequence[i:i+n])
        prefix = ngram[:-1]
        word = ngram[-1]
        p = count[prefix][word] / max(total[prefix], 1)
        if log:
            print("p(%s | %s) =\t%.3f" % (word, ', '.join(prefix), p))

        total_p *= p
    return total_p
    

print("\nProduct: p(%s) = %.3f" % (''.join(sequence[1:-1]), sequence_p(sequence, log=True)))

p(A | <bos>) =	1.000
p(A | A) =	0.615
p(B | A) =	0.231
p(<eos> | B) =	0.500

Product: p(AAB) = 0.071


### Real Example: Dialogue Utterances

Now lets use the same ideas on a more realistic text corpus.

We will use utterances from a dialogue dataset called **Persona-Chat**. This dataset is relatively small and centers on a single domain, but it is simple and interpretable for our purposes here.

We'll also use an off-the-shelf ngram modeling package called `KenLM`.

#### Loading the dataset

In [9]:
train = []
valid = []
import jsonlines

for ds, name in [(train, 'train'), (valid, 'valid')]:
    for line in jsonlines.Reader(open('data/personachat/personachat_all_sentences_%s.jsonl' % name, 'r')):
        ds.append(line['tokens'])
        
vocab = list(set([t for ts in train for t in ts]))      
print("Number of train examples: %d" % (len(train)))
print("Number of valid examples: %d" % (len(valid)))
print("Vocab size: %d" % (len(vocab)))

print("\nExamples:")
pprint(train[:3])

Number of train examples: 133176
Number of valid examples: 16181
Vocab size: 19153

Examples:
[['i', 'am', 'doing', 'great', 'except', 'for', 'the', 'allergies', '.'],
 ['i', 'am', 'a', 'woman', 'what', 'about', 'you', '.'],
 ['i', 'thought', 'you', 'were', 'a', 'college', 'kid', '.']]


### KenLM

KenLM estimates n-gram language models using **modified Kneser-Ney smoothing**, and has a fast and memory-efficient implementation. 
- While we won't go into details here, **smoothing** is a technique used to account for ngrams that do not occur in the training corpus. 
- Normally, these ngrams would receive zero-probability mass. Smoothing ensures every ngram receives some probability.



Please see page 48 of the [lecture note](https://github.com/nyu-dl/NLP_DL_Lecture_Note/blob/master/lecture_note.pdf) for an overview of Kneser-Ney smoothing, and [[Chen & Goodman 1998]](https://dash.harvard.edu/bitstream/handle/1/25104739/tr-10-98.pdf?sequence=1) for further details.

In [10]:
KENLM_DIR='/home/jupyter/kenlm'

#### Tokenize data

In [11]:
if not os.path.exists('data/tokenized'):
    os.makedirs('data/tokenized')
with open('data/tokenized/pchat_train', 'w') as f:
    for line in tqdm(train):
        f.write(' '.join(line))
        f.write('\n')

100%|██████████| 133176/133176 [00:00<00:00, 795531.18it/s]


#### Build kenlm n-gram models

This uses the `kenlm` commands `lmplz` to estimate the language model, then `build_binary` to convert it to an efficient format. We load the resulting model using the `kenlm` python wrapper.

We do this for n-gram models of order `2,3,4`:

In [17]:
import kenlm

data_prefix = 'pchat'
dataset = 'pchat_train'
if not os.path.exists('models'):
    os.makedirs('models')

models = {}
for n in [2,3,4]:
    model_temp = 'models/%s_%d.arpa' % (data_prefix, n)
    model_name = 'models/%s_%d.klm' % (data_prefix, n)
    ! cat ./data/tokenized/$dataset | $KENLM_DIR/build/bin/lmplz -o $n > $model_temp
    ! $KENLM_DIR/build/bin/build_binary $model_temp $model_name
    models[n] = kenlm.LanguageModel(model_name)

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 1602042 types 19156
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:229872 2:10928379904
Statistics:
1 19156 D1=0.588857 D2=1.0348 D3+=1.3388
2 231267 D1=0.708637 D2=1.06132 D3+=1.37629
Memory estimate for binary LM:
type      kB
probing 4551 assuming -p 1.5
probing 4626 assuming -r models -p 1.5
trie    1747 without quantization
trie    1099 assuming -q 8 -b 8 quantization 
trie    1747 assuming -a 22 array pointer compression
trie    1099 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:229872 2:3700272
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
####################################################################################################
=== 4/5 Calculating and writing order

## Evaluation

### Perplexity

Intuitively, a good model should assign high probabilities to sequences from the 'true' distribution that it is modeling.

A common way of quantifying this is with **perplexity**, a metric inversely-proportional to the probability that the model assigns to a set of sequences, e.g. a 'test set':

\begin{align}
\large \text{ppl}(p, D) &\large\ = 2^{-\frac{1}{N_{total}}\log_2 p(D)}
\end{align}


where $D=\{(w_1,\ldots,w_{N_i})_i\}_{i=1}^M$ is a dataset of $M$ sequences with total length $N_{\text{total}}=\sum_{i}N_i$.

Perplexity is defined on $[1,\infty)$, with 1 being a perfect model (assigning probability 1 to $D$), and a 'worse' model as perplexity increases.

Intuitively, _perplexity measures the average rank of the true next-token, when tokens are ordered by the model's conditional probabilities_.


<!-- Section 1.3 of [[Chen & Goodman 1998]](https://dash.harvard.edu/bitstream/handle/1/25104739/tr-10-98.pdf?sequence=1) has a concise summary of perplexity and its motivation. !-->

#### Evaluate Perplexity

`kenlm` outputs log probabilities in **log base 10**. For the standard definition of perplexity we need **log base 2**. See the code below:

In [13]:
def perplexity_kenlm(model, sequences):
    n_total = 0
    logp_total = 0
    for sequence in sequences:
        text = ' '.join(sequence)
        logp_total += model.score(text)
        n_total += len(sequence) + 1  # add 1 for </s>
        
    # Convert log10 to log2
    log2p_total = logp_total / np.log10(2)
    ppl = 2 ** (- (1.0 / n_total) * log2p_total)
    return ppl

In [14]:
print("=== Train ===")
df = pd.DataFrame([(n, perplexity_kenlm(models[n], train)) for n in [2, 3, 4]], columns=['n', 'ppl'])
df.style.hide_index()

=== Train ===


n,ppl
2,39.9248
3,15.462
4,8.91961


In [15]:
print("=== Valid ===")
df = pd.DataFrame([(n, perplexity_kenlm(models[n], valid)) for n in [2, 3, 4]], columns=['n', 'ppl'])
df.style.hide_index()

=== Valid ===


n,ppl
2,59.188
3,44.9771
4,43.2188


#### Sequence probabilities:
\begin{align}
p(w_1,\ldots,w_T)&\approx\prod_{t=1}^T p(w_t|w_{t-2},w_{t-1})\\
&=\sum_{t=1}^T \log p(w_t|w_{t-2},w_{t-1}).
\end{align}

where we use log probabilities in practice to avoid a product of many small numbers.

In [16]:
sentences = [
    'i like my pet dog .',
    'i like my pet zebra .',
    'i like my pet lion .',
    'i live in the united states .',
    'i live in the united states of america .'
]

for sentence in sentences:
    print(sentence)
    for n in [2, 3, 4]:
        log10p = models[n].score(sentence)
        log2p = log10p / np.log10(2)
        print("n: %d\t logp: %.3f" % (n, log2p))
    print()

i like my pet dog .
n: 2	 logp: -29.216
n: 3	 logp: -28.843
n: 4	 logp: -29.219

i like my pet zebra .
n: 2	 logp: -32.303
n: 3	 logp: -31.157
n: 4	 logp: -32.101

i like my pet lion .
n: 2	 logp: -38.352
n: 3	 logp: -41.017
n: 4	 logp: -42.320

i live in the united states .
n: 2	 logp: -22.599
n: 3	 logp: -20.095
n: 4	 logp: -17.854

i live in the united states of america .
n: 2	 logp: -40.958
n: 3	 logp: -26.627
n: 4	 logp: -24.311

